# 🇫🇷 Fine-tuning Vigostral-7B-Chat sur votre style personnel

Ce notebook vous guide pour fine-tuner le modèle conversationnel français **Vigostral-7B-Chat** sur vos dialogues personnels, en utilisant **LoRA** (Low-Rank Adaptation) pour s'adapter aux contraintes mémoire du GPU T4 gratuit de Google Colab.

## 📋 Prérequis

1. **GPU T4 activé** : Runtime → Change runtime type → T4 GPU
2. **Dataset de dialogues** au format JSONL
3. **Compte HuggingFace** (gratuit) pour télécharger Vigostral

## 🎯 Objectif

Adapter Vigostral-7B-Chat (modèle conversationnel français pré-entraîné sur 213k dialogues) à votre style d'écriture personnel en fine-tunant sur vos propres conversations.

## ⚙️ Technique : LoRA (Low-Rank Adaptation)

- **Problème** : Fine-tuner 7B paramètres nécessite ~50GB de VRAM (impossible sur T4)
- **Solution** : LoRA ne fine-tune que ~1% des paramètres (~70M)
- **Avantage** : Tient dans 16GB, training rapide, qualité préservée


---

## 📦 Installation des dépendances

On installe les bibliothèques nécessaires :
- `transformers` : Charger Vigostral depuis HuggingFace
- `peft` : Implémenter LoRA
- `accelerate` : Optimiser l'entraînement GPU
- `bitsandbytes` : Quantization 4-bit pour réduire la mémoire
- `trl` : Outils de fine-tuning pour LLMs


In [ ]:
!pip install -q transformers peft accelerate bitsandbytes trl datasets

---

## 🔑 Authentification HuggingFace

Vigostral-7B-Chat nécessite un token HuggingFace pour télécharger le modèle.

1. Allez sur https://huggingface.co/settings/tokens
2. Créez un token (Read access suffit)
3. Collez-le ci-dessous quand demandé


In [ ]:
from huggingface_hub import login
login()

---

## 📤 Upload de votre dataset

Uploadez votre fichier `combined_dataset.jsonl` (format : une ligne par dialogue)


In [ ]:
from google.colab import files
import os

print("📤 Uploadez votre fichier combined_dataset.jsonl")
uploaded = files.upload()

# Vérifier que le fichier est bien uploadé
if 'combined_dataset.jsonl' not in uploaded:
    raise FileNotFoundError("Fichier combined_dataset.jsonl non trouvé. Réessayez l'upload.")

print("✅ Dataset uploadé avec succès !")

---

## 📊 Chargement et validation du dataset


In [ ]:
import json

# Charger le dataset
dialogues = []
with open('combined_dataset.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        dialogues.append(json.loads(line))

print(f"✅ {len(dialogues)} dialogues chargés")

# Afficher un exemple
print("\n📋 Exemple de dialogue :")
print(json.dumps(dialogues[0], indent=2, ensure_ascii=False))

---

## 🔄 Conversion au format Vigostral

Vigostral utilise un format de prompt spécifique. On doit convertir nos dialogues.


In [ ]:
def format_dialogue_for_vigostral(dialogue):
    """
    Convertit un dialogue du format OpenAI au format Vigostral.
    
    Vigostral utilise le format Llama-2 :
    <s>[INST] question [/INST] réponse </s>
    """
    messages = dialogue['messages']
    formatted_text = "<s>"
    
    for i in range(0, len(messages), 2):
        if i < len(messages) and messages[i]['role'] == 'user':
            user_msg = messages[i]['content']
            formatted_text += f"[INST] {user_msg} [/INST]"
        
        if i + 1 < len(messages) and messages[i + 1]['role'] == 'assistant':
            assistant_msg = messages[i + 1]['content']
            formatted_text += f" {assistant_msg} </s>"
    
    return formatted_text

# Convertir tous les dialogues
formatted_dialogues = [format_dialogue_for_vigostral(d) for d in dialogues]

print(f"✅ {len(formatted_dialogues)} dialogues formatés")
print("\n📋 Exemple de dialogue formaté :")
print(formatted_dialogues[0][:500] + "...")

---

## 🤖 Chargement du modèle Vigostral-7B-Chat

On charge le modèle avec :
- **Quantization 4-bit** : Réduit la taille de ~14GB à ~4GB
- **LoRA** : Ajoute des adaptateurs entraînables (~70M params)


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_name = "bofenghuang/vigostral-7b-chat"

# Configuration de la quantization 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("📥 Téléchargement du modèle Vigostral-7B-Chat (peut prendre 5-10 minutes)...")

# Charger le modèle en 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Modèle chargé avec succès !")
print(f"   Taille du modèle : {model.get_memory_footprint() / 1e9:.2f} GB")

---

## 🎛️ Configuration LoRA

LoRA ajoute de petites matrices entraînables aux couches d'attention du modèle.

**Paramètres** :
- `r=16` : Rang des matrices (plus haut = plus de capacité, mais plus de mémoire)
- `lora_alpha=32` : Facteur de scaling
- `target_modules` : Couches à adapter (query et value projections)


In [ ]:
# Préparer le modèle pour LoRA
model = prepare_model_for_kbit_training(model)

# Configuration LoRA
lora_config = LoraConfig(
    r=16,  # Rang des matrices LoRA
    lora_alpha=32,  # Facteur de scaling
    target_modules=["q_proj", "v_proj"],  # Couches à adapter
    lora_dropout=0.05,  # Dropout pour régularisation
    bias="none",
    task_type="CAUSAL_LM",
)

# Appliquer LoRA au modèle
model = get_peft_model(model, lora_config)

# Afficher le nombre de paramètres entraînables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f"✅ LoRA activé !")
print(f"   Paramètres entraînables : {trainable_params:,} ({100 * trainable_params / all_params:.2f}%)")
print(f"   Paramètres totaux : {all_params:,}")

---

## 📚 Préparation du dataset pour l'entraînement


In [ ]:
from datasets import Dataset

# Créer un dataset HuggingFace
dataset_dict = {"text": formatted_dialogues}
dataset = Dataset.from_dict(dataset_dict)

# Split train/validation (90/10)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"✅ Dataset préparé :")
print(f"   Train : {len(dataset['train'])} dialogues")
print(f"   Validation : {len(dataset['test'])} dialogues")

---

## 🚀 Configuration de l'entraînement

**Paramètres optimisés pour T4** :
- `per_device_train_batch_size=1` : 1 exemple à la fois (limite mémoire)
- `gradient_accumulation_steps=4` : Accumule 4 gradients avant mise à jour (batch effectif = 4)
- `num_train_epochs=3` : 3 passages sur le dataset
- `learning_rate=2e-4` : Learning rate pour LoRA
- `fp16=True` : Utilise float16 pour économiser la mémoire


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./vigostral-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    logging_steps=10,
    optim="paged_adamw_8bit",  # Optimizer optimisé pour mémoire
    warmup_steps=50,
    report_to="none",  # Désactiver wandb/tensorboard
)

# Créer le trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=512,
)

print("✅ Trainer configuré !")

---

## 🎯 Lancement de l'entraînement

Durée estimée : **20-30 minutes** sur T4 GPU


In [ ]:
import time

print("🚀 Début de l'entraînement...\n")
start_time = time.time()

trainer.train()

elapsed_time = time.time() - start_time
print(f"\n✅ Entraînement terminé en {elapsed_time / 60:.1f} minutes !")

---

## 💾 Sauvegarde du modèle fine-tuné


In [ ]:
# Sauvegarder le modèle et les adaptateurs LoRA
model.save_pretrained("./vigostral-finetuned-final")
tokenizer.save_pretrained("./vigostral-finetuned-final")

print("✅ Modèle sauvegardé dans ./vigostral-finetuned-final")

---

## 🧪 Test du modèle fine-tuné

Testons le modèle avec quelques prompts pour voir comment il répond dans votre style.


In [ ]:
def chat(prompt, max_length=200, temperature=0.7, top_p=0.9):
    """
    Génère une réponse du modèle fine-tuné.
    """
    # Formater le prompt au format Vigostral
    formatted_prompt = f"<s>[INST] {prompt} [/INST]"
    
    # Tokenizer
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    # Générer
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Décoder la réponse
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraire seulement la réponse (après [/INST])
    if "[/INST]" in response:
        response = response.split("[/INST]")[1].strip()
    
    return response

# Test avec quelques prompts
test_prompts = [
    "Explique-moi ton projet principal",
    "Qu'est-ce que tu penses du machine learning ?",
    "Comment tu travailles au quotidien ?",
]

print("\n🧪 Test du modèle fine-tuné :\n")
print("=" * 80)

for prompt in test_prompts:
    print(f"\n👤 User: {prompt}")
    response = chat(prompt)
    print(f"🤖 Assistant: {response}")
    print("-" * 80)

---

## 💬 Mode interactif

Testez librement votre modèle !


In [ ]:
print("💬 Mode chat interactif")
print("Tapez 'quit' pour quitter\n")

while True:
    user_input = input("👤 Vous: ")
    
    if user_input.lower() in ['quit', 'exit', 'q']:
        print("👋 Au revoir !")
        break
    
    if not user_input.strip():
        continue
    
    response = chat(user_input)
    print(f"🤖 Assistant: {response}\n")

---

## 📦 Télécharger le modèle fine-tuné

Pour utiliser votre modèle localement, téléchargez les fichiers sauvegardés.


In [ ]:
# Créer une archive zip du modèle
!zip -r vigostral-finetuned-final.zip vigostral-finetuned-final/

# Télécharger l'archive
from google.colab import files
files.download('vigostral-finetuned-final.zip')

print("✅ Modèle téléchargé ! Taille des adaptateurs LoRA : ~100-200 MB")

---

## 🎉 Félicitations !

Vous avez fine-tuné Vigostral-7B-Chat sur votre style personnel !

### 📝 Prochaines étapes

1. **Tester localement** :
   ```python
   from peft import PeftModel
   from transformers import AutoModelForCausalLM, AutoTokenizer
   
   # Charger le modèle de base
   model = AutoModelForCausalLM.from_pretrained("bofenghuang/vigostral-7b-chat")
   
   # Charger les adaptateurs LoRA
   model = PeftModel.from_pretrained(model, "./vigostral-finetuned-final")
   ```

2. **Intégrer avec nanochat** : Voir documentation séparée

3. **Améliorer le modèle** :
   - Ajouter plus de dialogues (200+ recommandé)
   - Augmenter les epochs (5-10)
   - Ajuster `r` dans LoRA (16 → 32 pour plus de capacité)

---

**Questions ?** Ouvrez une issue sur GitHub : https://github.com/cladjidane/nanochat-french-tutorial